# RESEARCH DATA

## Modules & Libraries

In [ ]:
import pandas as pd
import numpy as np
import pathlib

In [ ]:
import time
import datetime as dt
#from datetime import date, timedelta

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Structure df

In [ ]:
df_struct = pd.DataFrame()

In [ ]:
start_date = dt.date(2020, 1, 1)
end_date = dt.date(2023, 6, 30)    # perhaps date.now()

delta = end_date - start_date   # returns timedelta

In [ ]:
i = 0

for x in range(delta.days + 1):

  day = start_date + dt.timedelta(days=x)

  for y in range(1,49):

    df_struct.loc[i, 'settlementDate'] = day
    df_struct.loc[i, 'settlementPeriod'] = y

    i += 1


In [ ]:
df_struct['settlementDate'] = pd.to_datetime(df_struct['settlementDate'])
df_struct

,settlementDate,settlementPeriod
0,2020-01-01,1.0
1,2020-01-01,2.0
2,2020-01-01,3.0
3,2020-01-01,4.0
4,2020-01-01,5.0
...,...,...
61291,2023-06-30,44.0
61292,2023-06-30,45.0
61293,2023-06-30,46.0
61294,2023-06-30,47.0


## Loading datasets

### - UK Holidays

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Data/Research/UK Holidays/UK_Holidays_Angular.xlsx'

In [ ]:
df_01 = pd.read_excel(file_path)
df_01

,Dates,Day of the Week,Weekday Name,Weekend,Holiday,Angle,Angular Difference
0,2020-12-20,6,Sunday,1,0,NaN,NaN
1,2020-12-21,0,Monday,0,0,NaN,NaN
2,2020-12-22,1,Tuesday,0,0,NaN,NaN
3,2020-12-23,2,Wednesday,0,0,NaN,NaN
4,2020-12-24,3,Thursday,0,0,NaN,NaN
...,...,...,...,...,...,...,...
918,2023-06-26,0,Monday,0,0,0.000000,5.385587
919,2023-06-27,1,Tuesday,0,0,0.897598,0.897598
920,2023-06-28,2,Wednesday,0,0,1.795196,0.897598
921,2023-06-29,3,Thursday,0,0,2.692794,0.897598


In [ ]:
df_01 = df_01.loc[:,['Dates', 'Weekday Name', 'Weekend', 'Holiday']]
df_01

,Dates,Weekday Name,Weekend,Holiday
0,2020-12-20,Sunday,1,0
1,2020-12-21,Monday,0,0
2,2020-12-22,Tuesday,0,0
3,2020-12-23,Wednesday,0,0
4,2020-12-24,Thursday,0,0
...,...,...,...,...
918,2023-06-26,Monday,0,0
919,2023-06-27,Tuesday,0,0
920,2023-06-28,Wednesday,0,0
921,2023-06-29,Thursday,0,0


#### Adding independent 'Year', 'Month' and 'Day' columns

In [ ]:
df_01['Year'] = df_01['Dates'].dt.year
df_01['Month'] = df_01['Dates'].dt.month
df_01['Day'] = df_01['Dates'].dt.day
df_01 = df_01.rename(columns={'Weekday Name':'Weekday'})

In [ ]:
df_01

,Dates,Weekday,Weekend,Holiday,Year,Month,Day
0,2020-12-20,Sunday,1,0,2020,12,20
1,2020-12-21,Monday,0,0,2020,12,21
2,2020-12-22,Tuesday,0,0,2020,12,22
3,2020-12-23,Wednesday,0,0,2020,12,23
4,2020-12-24,Thursday,0,0,2020,12,24
...,...,...,...,...,...,...,...
918,2023-06-26,Monday,0,0,2023,6,26
919,2023-06-27,Tuesday,0,0,2023,6,27
920,2023-06-28,Wednesday,0,0,2023,6,28
921,2023-06-29,Thursday,0,0,2023,6,29


#### Categoric cycle variables

In [ ]:
def cicl_tranf_diasem(df_input, col_diasem):

    nrows = df_input.shape[0]
    df_out = df_input.copy()
    df_out["Weekday_SIN"] = ""
    df_out["Weekday_COS"] = ""

    for i in range(0,nrows):
        dias_sem = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
        dia_sem = df_input[col_diasem][i]
        value_dia_sem = dias_sem[dia_sem]

        df_out.at[i, 'Weekday_SIN'] = np.sin((value_dia_sem-1)*(2.*np.pi/7))
        df_out.at[i, 'Weekday_COS'] = np.cos((value_dia_sem-1)*(2.*np.pi/7))

    df_out = df_out.astype({'Weekday_SIN':'float64', 'Weekday_COS':'float64'})

    return df_out

In [ ]:
df_01 = cicl_tranf_diasem(df_input=df_01, col_diasem='Weekday')

In [ ]:
def cicl_tranf_month(df_input, col_month):

    nrows = df_input.shape[0]
    df_out = df_input.copy()
    df_out["Month_SIN"] = ""
    df_out["Month_COS"] = ""

    for i in range(0,nrows):
        month_i = df_input[col_month][i]

        df_out.at[i, 'Month_SIN'] = np.sin((month_i-1)*(2.*np.pi/12))
        df_out.at[i, 'Month_COS'] = np.cos((month_i-1)*(2.*np.pi/12))

    df_out = df_out.astype({'Month_SIN':'float64', 'Month_COS':'float64'})

    return df_out

In [ ]:
df_01 = cicl_tranf_month(df_input=df_01, col_month='Month')

In [ ]:
def cicl_tranf_day(df_input, col_day, col_month, col_ano):

    nrows = df_input.shape[0]
    df_out = df_input.copy()
    df_out["Day_SIN"] = ""
    df_out["Day_COS"] = ""

    for i in range(0,nrows):
        day_i = df_input[col_day][i]
        month_i = df_input[col_month][i]
        ano_i = df_input[col_ano][i]

        month_31 = [1,3,5,7,8,10,12]
        month_30 = [4,6,9,11]
        month_28_29 = [2]

        ano_29 = []
        for x in range(2012,2500,4):
            ano_29.append(x)

        if month_i in month_31:
            df_out.at[i, 'Day_SIN'] = np.sin((day_i-1)*(2.*np.pi/31))
            df_out.at[i, 'Day_COS'] = np.cos((day_i-1)*(2.*np.pi/31))
        if month_i in month_30:
            df_out.at[i, 'Day_SIN'] = np.sin((day_i-1)*(2.*np.pi/30))
            df_out.at[i, 'Day_COS'] = np.cos((day_i-1)*(2.*np.pi/30))
        if month_i in month_28_29 and ano_i not in ano_29:
            df_out.at[i, 'Day_SIN'] = np.sin((day_i-1)*(2.*np.pi/28))
            df_out.at[i, 'Day_COS'] = np.cos((day_i-1)*(2.*np.pi/28))
        if month_i in month_28_29 and ano_i in ano_29:
            df_out.at[i, 'Day_SIN'] = np.sin((day_i-1)*(2.*np.pi/29))
            df_out.at[i, 'Day_COS'] = np.cos((day_i-1)*(2.*np.pi/29))

    df_out = df_out.astype({'Day_SIN':'float64', 'Day_COS':'float64'})

    return df_out

In [ ]:
df_01 = cicl_tranf_day(df_input=df_01, col_day='Day', col_month='Month', col_ano='Year')

In [ ]:
df_01

,Dates,Weekday,Weekend,Holiday,Year,Month,Day,Weekday_SIN,Weekday_COS,Month_SIN,Month_COS,Day_SIN,Day_COS
0,2020-12-20,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
1,2020-12-21,Monday,0,0,2020,12,21,0.000000,1.000000,-0.5,0.866025,-0.790776,-0.612106
2,2020-12-22,Tuesday,0,0,2020,12,22,0.781831,0.623490,-0.5,0.866025,-0.897805,-0.440394
3,2020-12-23,Wednesday,0,0,2020,12,23,0.974928,-0.222521,-0.5,0.866025,-0.968077,-0.250653
4,2020-12-24,Thursday,0,0,2020,12,24,0.433884,-0.900969,-0.5,0.866025,-0.998717,-0.050649
...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,2023-06-26,Monday,0,0,2023,6,26,0.000000,1.000000,0.5,-0.866025,-0.866025,0.500000
919,2023-06-27,Tuesday,0,0,2023,6,27,0.781831,0.623490,0.5,-0.866025,-0.743145,0.669131
920,2023-06-28,Wednesday,0,0,2023,6,28,0.974928,-0.222521,0.5,-0.866025,-0.587785,0.809017
921,2023-06-29,Thursday,0,0,2023,6,29,0.433884,-0.900969,0.5,-0.866025,-0.406737,0.913545


### - Elexon API

#### General data

In [ ]:
folder_path = '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data'

In [ ]:
desktop = pathlib.Path(folder_path)
# .iterdir() produces a generator
desktop.iterdir()

<generator object Path.iterdir at 0x7b5660f55ee0>

In [ ]:
# Which you can wrap in a list() constructor to materialize
files_list = list(desktop.iterdir())
files_list

[PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.2.2 Bid Offer).csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.2.8 Market Index Data).csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/DataFrame Structure.csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.1.13 B0620).csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.1.20 B1430).csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (04_5.2.29 Margin & Imbalance).csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.1.21 B1440).csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.2.30 Demand).csv'),
 PosixPath('/content/drive/MyD

In [ ]:
files_list = ['/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/DataFrame Structure.csv',
              '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.2.8 Market Index Data).csv',
              '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.2.30 Demand).csv',
              '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.1.13 B0620).csv',
              '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (04_5.2.29 Margin & Imbalance).csv',
              '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.1.20 B1430).csv',
              '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Elexon Final Data (5.1.21 B1440).csv']

In [ ]:
for x in files_list:
  df_aux = pd.read_csv(x)

  if files_list.index(x) == 0:
    df_02 = df_aux
  else:
    df_02 = pd.merge(df_02, df_aux, on=['settlementDate','settlementPeriod'])

In [ ]:
df_02

,settlementDate,settlementPeriod,recordType,marketIndexDataProviderId,marketIndexPrice,marketIndexVolume,activeFlag,DANF demand,DAID spnDemand,DAIG spnGeneration,DATF demand,System total load Consumption,DAM,DAI,Generation forecast Production,Solar generation,Wind Offshore generation,Wind Onshore generation
0,2020-12-20,1.0,MID,APXMIDP,20.65,894.75,True,22700.0,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,NaN,0.0,9020.590,7365.717
1,2020-12-20,2.0,MID,APXMIDP,25.34,672.75,True,22300.0,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,NaN,0.0,9020.590,7365.717
2,2020-12-20,3.0,MID,APXMIDP,35.97,565.95,True,21865.0,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.0,0.0,8871.388,7352.576
3,2020-12-20,4.0,MID,APXMIDP,38.93,794.25,True,21214.0,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.0,0.0,8871.388,7352.576
4,2020-12-20,5.0,MID,APXMIDP,39.03,685.45,True,20772.0,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.0,0.0,8670.041,7312.832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44018,2023-06-30,44.0,MID,APXMIDP,83.77,1638.70,True,24428.0,-16510.0,26897.0,25613.0,26127.0,21172.0,1284.0,27783.0,0.0,8815.046,4170.762
44019,2023-06-30,45.0,MID,APXMIDP,87.95,1848.10,True,23533.0,-16426.0,25785.0,25123.0,25102.0,21041.0,662.0,27868.0,0.0,9051.941,4506.462
44020,2023-06-30,46.0,MID,APXMIDP,85.35,2492.25,True,22272.0,-15751.0,24347.0,23862.0,23707.0,22087.0,485.0,27445.0,0.0,9051.941,4506.462
44021,2023-06-30,47.0,MID,APXMIDP,53.20,1380.95,True,21015.0,-14830.0,21727.0,22321.0,22499.0,22968.0,-594.0,24550.0,0.0,9152.405,4949.377


In [ ]:
# "DATE" COLUMN
df_02['settlementDate'] = pd.to_datetime(df_02['settlementDate'])

In [ ]:
df_02.columns

Index(['settlementDate', 'settlementPeriod', 'recordType',
       'marketIndexDataProviderId', 'marketIndexPrice', 'marketIndexVolume',
       'activeFlag', 'DANF demand', 'DAID spnDemand', 'DAIG spnGeneration',
       'DATF demand', 'System total load Consumption', 'DAM', 'DAI',
       'Generation forecast Production', 'Solar generation',
       'Wind Offshore generation', 'Wind Onshore generation'],
      dtype='object')

In [ ]:
df_02 = df_02.drop(columns=['recordType',	'marketIndexDataProviderId', 'activeFlag'], axis=1)

df_02 = df_02.rename(columns={'DANF demand':'DANF_Demand', 'DAID spnDemand':'DAID_spnDemand',
                              'DAIG spnGeneration':'DAIG_spnGeneration', 'DATF demand':'DATF_Demand', 'Solar generation':'Solar_Generation',
                              'Wind Offshore generation':'WindOffshore_Generation', 'Wind Onshore generation':'WindOnshore_Generation',
                              'Generation forecast Production':'GenerationForecast_Production', 'System total load Consumption':'SystemTotalLoad_Consumption',
                              'DAM':'DAMargin', 'DAI':'DAImbalance'})

In [ ]:
df_02

,settlementDate,settlementPeriod,marketIndexPrice,marketIndexVolume,DANF_Demand,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,2020-12-20,1.0,20.65,894.75,22700.0,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,NaN,0.0,9020.590,7365.717
1,2020-12-20,2.0,25.34,672.75,22300.0,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,NaN,0.0,9020.590,7365.717
2,2020-12-20,3.0,35.97,565.95,21865.0,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.0,0.0,8871.388,7352.576
3,2020-12-20,4.0,38.93,794.25,21214.0,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.0,0.0,8871.388,7352.576
4,2020-12-20,5.0,39.03,685.45,20772.0,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.0,0.0,8670.041,7312.832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44018,2023-06-30,44.0,83.77,1638.70,24428.0,-16510.0,26897.0,25613.0,26127.0,21172.0,1284.0,27783.0,0.0,8815.046,4170.762
44019,2023-06-30,45.0,87.95,1848.10,23533.0,-16426.0,25785.0,25123.0,25102.0,21041.0,662.0,27868.0,0.0,9051.941,4506.462
44020,2023-06-30,46.0,85.35,2492.25,22272.0,-15751.0,24347.0,23862.0,23707.0,22087.0,485.0,27445.0,0.0,9051.941,4506.462
44021,2023-06-30,47.0,53.20,1380.95,21015.0,-14830.0,21727.0,22321.0,22499.0,22968.0,-594.0,24550.0,0.0,9152.405,4949.377


In [ ]:
df_02.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44023 entries, 0 to 44022
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   settlementDate                 44023 non-null  datetime64[ns]
 1   settlementPeriod               44023 non-null  float64       
 2   marketIndexPrice               44023 non-null  float64       
 3   marketIndexVolume              44023 non-null  float64       
 4   DANF_Demand                    44023 non-null  float64       
 5   DAID_spnDemand                 44023 non-null  float64       
 6   DAIG_spnGeneration             44023 non-null  float64       
 7   DATF_Demand                    44023 non-null  float64       
 8   SystemTotalLoad_Consumption    43349 non-null  float64       
 9   DAMargin                       44023 non-null  float64       
 10  DAImbalance                    44023 non-null  float64       
 11  GenerationForec

In [ ]:
df_02.isnull().sum()

settlementDate                     0
settlementPeriod                   0
marketIndexPrice                   0
marketIndexVolume                  0
DANF_Demand                        0
DAID_spnDemand                     0
DAIG_spnGeneration                 0
DATF_Demand                        0
SystemTotalLoad_Consumption      674
DAMargin                           0
DAImbalance                        0
GenerationForecast_Production    578
Solar_Generation                 624
WindOffshore_Generation          624
WindOnshore_Generation           624
dtype: int64

#### Adding Dates info

In [ ]:
df_aux = df_struct[df_struct['settlementDate']>='2020-12-20']
df_aux

,settlementDate,settlementPeriod
16992,2020-12-20,1.0
16993,2020-12-20,2.0
16994,2020-12-20,3.0
16995,2020-12-20,4.0
16996,2020-12-20,5.0
...,...,...
61291,2023-06-30,44.0
61292,2023-06-30,45.0
61293,2023-06-30,46.0
61294,2023-06-30,47.0


In [ ]:
df_01

,Dates,Weekday,Weekend,Holiday,Year,Month,Day,Weekday_SIN,Weekday_COS,Month_SIN,Month_COS,Day_SIN,Day_COS
0,2020-12-20,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
1,2020-12-21,Monday,0,0,2020,12,21,0.000000,1.000000,-0.5,0.866025,-0.790776,-0.612106
2,2020-12-22,Tuesday,0,0,2020,12,22,0.781831,0.623490,-0.5,0.866025,-0.897805,-0.440394
3,2020-12-23,Wednesday,0,0,2020,12,23,0.974928,-0.222521,-0.5,0.866025,-0.968077,-0.250653
4,2020-12-24,Thursday,0,0,2020,12,24,0.433884,-0.900969,-0.5,0.866025,-0.998717,-0.050649
...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,2023-06-26,Monday,0,0,2023,6,26,0.000000,1.000000,0.5,-0.866025,-0.866025,0.500000
919,2023-06-27,Tuesday,0,0,2023,6,27,0.781831,0.623490,0.5,-0.866025,-0.743145,0.669131
920,2023-06-28,Wednesday,0,0,2023,6,28,0.974928,-0.222521,0.5,-0.866025,-0.587785,0.809017
921,2023-06-29,Thursday,0,0,2023,6,29,0.433884,-0.900969,0.5,-0.866025,-0.406737,0.913545


In [ ]:
df_01 = pd.merge(df_aux, df_01, how='left', left_on=['settlementDate'], right_on=['Dates'], suffixes=('','')).drop(columns='Dates', axis=1)
df_01

,settlementDate,settlementPeriod,Weekday,Weekend,Holiday,Year,Month,Day,Weekday_SIN,Weekday_COS,Month_SIN,Month_COS,Day_SIN,Day_COS
0,2020-12-20,1.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
1,2020-12-20,2.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
2,2020-12-20,3.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
3,2020-12-20,4.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
4,2020-12-20,5.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,-0.5,0.866025,-0.651372,-0.758758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,0.5,-0.866025,-0.207912,0.978148
44300,2023-06-30,45.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,0.5,-0.866025,-0.207912,0.978148
44301,2023-06-30,46.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,0.5,-0.866025,-0.207912,0.978148
44302,2023-06-30,47.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,0.5,-0.866025,-0.207912,0.978148


In [ ]:
df_01_02 = pd.merge(df_01, df_02, how='left', on=['settlementDate', 'settlementPeriod'], suffixes=('',''))
df_01_02

,settlementDate,settlementPeriod,Weekday,Weekend,Holiday,Year,Month,Day,Weekday_SIN,Weekday_COS,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,2020-12-20,1.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,NaN,0.0,9020.590,7365.717
1,2020-12-20,2.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,NaN,0.0,9020.590,7365.717
2,2020-12-20,3.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.0,0.0,8871.388,7352.576
3,2020-12-20,4.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.0,0.0,8871.388,7352.576
4,2020-12-20,5.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.0,0.0,8670.041,7312.832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-16510.0,26897.0,25613.0,26127.0,21172.0,1284.0,27783.0,0.0,8815.046,4170.762
44300,2023-06-30,45.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-16426.0,25785.0,25123.0,25102.0,21041.0,662.0,27868.0,0.0,9051.941,4506.462
44301,2023-06-30,46.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-15751.0,24347.0,23862.0,23707.0,22087.0,485.0,27445.0,0.0,9051.941,4506.462
44302,2023-06-30,47.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-14830.0,21727.0,22321.0,22499.0,22968.0,-594.0,24550.0,0.0,9152.405,4949.377


In [ ]:
def cicl_tranf_settPeriod(df_input, col_settPeriod):

    nrows = df_input.shape[0]
    df_out = df_input.copy()
    df_out["settPeriod_SIN"] = ""
    df_out["settPeriod_COS"] = ""

    for i in range(0,nrows):
        settPeriod_i = df_input[col_settPeriod][i]

        df_out.at[i, 'settPeriod_SIN'] = np.sin((settPeriod_i-1)*(2.*np.pi/48))
        df_out.at[i, 'settPeriod_COS'] = np.cos((settPeriod_i-1)*(2.*np.pi/48))

    df_out = df_out.astype({'settPeriod_SIN':'float64', 'settPeriod_COS':'float64'})

    return df_out

In [ ]:
df_01_02 = cicl_tranf_settPeriod(df_input=df_01_02, col_settPeriod='settlementPeriod')

In [ ]:
df_01_02.insert(14, 'settPeriod_SIN', df_01_02.pop('settPeriod_SIN'))

In [ ]:
df_01_02.insert(15, 'settPeriod_COS', df_01_02.pop('settPeriod_COS'))

In [ ]:
#df_01_02[df_01_02['settlementDate']=='2021-01-04']

In [ ]:
df_01_02.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44304 entries, 0 to 44303
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   settlementDate                 44304 non-null  datetime64[ns]
 1   settlementPeriod               44304 non-null  float64       
 2   Weekday                        44304 non-null  object        
 3   Weekend                        44304 non-null  int64         
 4   Holiday                        44304 non-null  int64         
 5   Year                           44304 non-null  int64         
 6   Month                          44304 non-null  int64         
 7   Day                            44304 non-null  int64         
 8   Weekday_SIN                    44304 non-null  float64       
 9   Weekday_COS                    44304 non-null  float64       
 10  Month_SIN                      44304 non-null  float64       
 11  Month_COS      

In [ ]:
file_path = '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Complete Dataframe.csv'

In [ ]:
df_01_02.to_csv(file_path, index=False)

In [ ]:
df_01_02 = pd.read_csv(file_path)

#### Imputation (Miss Forest)

In [ ]:
file_path = "C:/Users/FAL/Downloads/Complete Dataframe.csv"

In [ ]:
df_01_02 = pd.read_csv(file_path)

In [ ]:
df_01_02

,settlementDate,settlementPeriod,Weekday,Weekend,Holiday,Year,Month,Day,Weekday_SIN,Weekday_COS,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,2020-12-20,1.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,NaN,0.0,9020.590,7365.717
1,2020-12-20,2.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,NaN,0.0,9020.590,7365.717
2,2020-12-20,3.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.0,0.0,8871.388,7352.576
3,2020-12-20,4.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.0,0.0,8871.388,7352.576
4,2020-12-20,5.0,Sunday,1,0,2020,12,20,-0.781831,0.623490,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.0,0.0,8670.041,7312.832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-16510.0,26897.0,25613.0,26127.0,21172.0,1284.0,27783.0,0.0,8815.046,4170.762
44300,2023-06-30,45.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-16426.0,25785.0,25123.0,25102.0,21041.0,662.0,27868.0,0.0,9051.941,4506.462
44301,2023-06-30,46.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-15751.0,24347.0,23862.0,23707.0,22087.0,485.0,27445.0,0.0,9051.941,4506.462
44302,2023-06-30,47.0,Friday,0,0,2023,6,30,-0.433884,-0.900969,...,-14830.0,21727.0,22321.0,22499.0,22968.0,-594.0,24550.0,0.0,9152.405,4949.377


In [ ]:
# DATE COLUMN
df_01_02['settlementDate'] = pd.to_datetime(df_01_02['settlementDate'])

# Transforming date to CATEGORICAL ORDINAL
order = list(df_01_02['settlementDate'].unique())
order.sort()
#print(order)
data = df_01_02['settlementDate']
df_01_02['settlementDate'] = pd.Categorical(data, order, ordered=True)


# CATEGORICAL ORDINAL

# Column "settlementPeriod"
order = list(df_01_02['settlementPeriod'].astype('int').unique())
order.sort()
print(order)
data = df_01_02['settlementPeriod']
df_01_02['settlementPeriod'] = pd.Categorical(data, order, ordered=True)

# Column "Weekday"
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print(order)
data = df_01_02['Weekday']
df_01_02['Weekday'] = pd.Categorical(data, order, ordered=True)

# Column "Year"
order = list(df_01_02['Year'].astype('int').unique())
order.sort()
print(order)
data = df_01_02['Year']
df_01_02['Year'] = pd.Categorical(data, order, ordered=True)

# Column "Month"
order = list(df_01_02['Month'].astype('int').unique())
order.sort()
print(order)
data = df_01_02['Month']
df_01_02['Month'] = pd.Categorical(data, order, ordered=True)

# Column "Day"
order = list(df_01_02['Day'].astype('int').unique())
order.sort()
print(order)
data = df_01_02['Day']
df_01_02['Day'] = pd.Categorical(data, order, ordered=True)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
[2020, 2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [ ]:
df_01_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44304 entries, 0 to 44303
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   settlementDate                 44304 non-null  category
 1   settlementPeriod               44304 non-null  category
 2   Weekday                        44304 non-null  category
 3   Weekend                        44304 non-null  int64   
 4   Holiday                        44304 non-null  int64   
 5   Year                           44304 non-null  category
 6   Month                          44304 non-null  category
 7   Day                            44304 non-null  category
 8   Weekday_SIN                    44304 non-null  float64 
 9   Weekday_COS                    44304 non-null  float64 
 10  Month_SIN                      44304 non-null  float64 
 11  Month_COS                      44304 non-null  float64 
 12  Day_SIN                        4

In [ ]:
# MISSING VALUES (BY COLUMNS)

df_data = df_01_02.copy()
df_colsnan = pd.DataFrame()

df_colsnan['dtypes'] = df_data.apply(lambda x: x.dtypes)
df_colsnan['nan'] = df_data.isnull().sum()
df_colsnan['perc_nan'] = (df_data.isnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_nan'] = df_colsnan['perc_nan'].round(2)
df_colsnan['not_nan'] = df_data.notnull().sum()
df_colsnan['perc_not_nan'] = (df_data.notnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_not_nan'] = df_colsnan['perc_not_nan'].round(2)
df_colsnan['unique_values'] = df_data.apply(lambda x: len(x.unique()) )
df_colsnan['neg_values'] = df_data.apply(lambda x: x.lt(0).sum() if x.dtypes == 'float64' else 'NA')
df_colsnan.reset_index(inplace=True)
df_colsnan.rename(columns={'index':'col_name'}, inplace=True)

df_colsnan

,col_name,dtypes,nan,perc_nan,not_nan,perc_not_nan,unique_values,neg_values
0,settlementDate,category,0,0.00,44304,100.00,923,NA
1,settlementPeriod,category,0,0.00,44304,100.00,48,NA
2,Weekday,category,0,0.00,44304,100.00,7,NA
3,Weekend,int64,0,0.00,44304,100.00,2,NA
4,Holiday,int64,0,0.00,44304,100.00,2,NA
5,Year,category,0,0.00,44304,100.00,4,NA
6,Month,category,0,0.00,44304,100.00,12,NA
7,Day,category,0,0.00,44304,100.00,31,NA
8,Weekday_SIN,float64,0,0.00,44304,100.00,7,18960
9,Weekday_COS,float64,0,0.00,44304,100.00,7,25296


In [ ]:
cat_vars = list(df_01_02.select_dtypes(exclude=["number"]).columns)
cat_vars

['settlementDate', 'settlementPeriod', 'Weekday', 'Year', 'Month', 'Day']

In [ ]:
num_vars = list(df_01_02.select_dtypes(include=["number"]).columns)
num_vars

['Weekend',
 'Holiday',
 'Weekday_SIN',
 'Weekday_COS',
 'Month_SIN',
 'Month_COS',
 'Day_SIN',
 'Day_COS',
 'settPeriod_SIN',
 'settPeriod_COS',
 'marketIndexPrice',
 'marketIndexVolume',
 'DANF_Demand',
 'DAID_spnDemand',
 'DAIG_spnGeneration',
 'DATF_Demand',
 'SystemTotalLoad_Consumption',
 'DAMargin',
 'DAImbalance',
 'GenerationForecast_Production',
 'Solar_Generation',
 'WindOffshore_Generation',
 'WindOnshore_Generation']

In [ ]:
df_01_02.columns

Index(['settlementDate', 'settlementPeriod', 'Weekday', 'Weekend', 'Holiday',
       'Year', 'Month', 'Day', 'Weekday_SIN', 'Weekday_COS', 'Month_SIN',
       'Month_COS', 'Day_SIN', 'Day_COS', 'settPeriod_SIN', 'settPeriod_COS',
       'marketIndexPrice', 'marketIndexVolume', 'DANF_Demand',
       'DAID_spnDemand', 'DAIG_spnGeneration', 'DATF_Demand',
       'SystemTotalLoad_Consumption', 'DAMargin', 'DAImbalance',
       'GenerationForecast_Production', 'Solar_Generation',
       'WindOffshore_Generation', 'WindOnshore_Generation'],
      dtype='object')

In [ ]:
cat_cols = df_01_02[cat_vars]
cat_cols = cat_cols.drop(columns=['settlementDate'], axis=1)
cat_cols.head()

,settlementPeriod,Weekday,Year,Month,Day
0,1,Sunday,2020,12,20
1,2,Sunday,2020,12,20
2,3,Sunday,2020,12,20
3,4,Sunday,2020,12,20
4,5,Sunday,2020,12,20


In [ ]:
num_cols = df_01_02[num_vars]
num_cols.head()

,Weekend,Holiday,Weekday_SIN,Weekday_COS,Month_SIN,Month_COS,Day_SIN,Day_COS,settPeriod_SIN,settPeriod_COS,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,1,0,-0.781831,0.62349,-0.5,0.866025,-0.651372,-0.758758,0.000000,1.000000,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,NaN,0.0,9020.590,7365.717
1,1,0,-0.781831,0.62349,-0.5,0.866025,-0.651372,-0.758758,0.130526,0.991445,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,NaN,0.0,9020.590,7365.717
2,1,0,-0.781831,0.62349,-0.5,0.866025,-0.651372,-0.758758,0.258819,0.965926,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.0,0.0,8871.388,7352.576
3,1,0,-0.781831,0.62349,-0.5,0.866025,-0.651372,-0.758758,0.382683,0.923880,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.0,0.0,8871.388,7352.576
4,1,0,-0.781831,0.62349,-0.5,0.866025,-0.651372,-0.758758,0.500000,0.866025,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.0,0.0,8670.041,7312.832


In [ ]:
ohe = OneHotEncoder(sparse=False)

In [ ]:
ohe.fit(cat_cols)

OneHotEncoder(sparse=False)

In [ ]:
cat_cols_values = ohe.transform(cat_cols)

In [ ]:
cat_cols_enc = pd.DataFrame(cat_cols_values)
cat_cols_enc

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
df_01_02_enc = cat_cols_enc.join(num_cols)
df_01_02_enc.head()

,0,1,2,3,4,5,6,7,8,9,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,NaN,0.0,9020.590,7365.717
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,NaN,0.0,9020.590,7365.717
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.0,0.0,8871.388,7352.576
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.0,0.0,8871.388,7352.576
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.0,0.0,8670.041,7312.832


In [ ]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [ ]:
#Miss Forest
imputer = MissForest(random_state=0)

In [ ]:
# Make an instance and perform the imputation
#df_01_02_imp = imputer.fit_transform(X=df_01_02)
df_01_02_imp = imputer.fit_transform(X=df_01_02_enc)

C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1

Iteration: 0


C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1

Iteration: 1


C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1

Iteration: 2


C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1

Iteration: 3


C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\MEDS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1

Iteration: 4


In [ ]:
df_01_02_imp

array([[1.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        9.020590e+03, 7.365717e+03],
       [0.000000e+00, 1.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        9.020590e+03, 7.365717e+03],
       [0.000000e+00, 0.000000e+00, 1.000000e+00, ..., 0.000000e+00,
        8.871388e+03, 7.352576e+03],
       ...,
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        9.051941e+03, 4.506462e+03],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        9.152405e+03, 4.949377e+03],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        9.152405e+03, 4.949377e+03]])

In [ ]:
df_01_02_imp = pd.DataFrame(df_01_02_imp, columns=df_01_02_enc.columns)
df_01_02_imp.head()

,0,1,2,3,4,5,6,7,8,9,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,25650.22,0.0,9020.590,7365.717
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,25229.70,0.0,9020.590,7365.717
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.00,0.0,8871.388,7352.576
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.00,0.0,8871.388,7352.576
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.00,0.0,8670.041,7312.832


In [ ]:
list(df_01_02_imp.columns)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 'Weekend',
 'Holiday',
 'Weekday_SIN',
 'Weekday_COS',
 'Month_SIN',
 'Month_COS',
 'Day_SIN',
 'Day_COS',
 'settPeriod_SIN',
 'settPeriod_COS',
 'marketIndexPrice',
 'marketIndexVolume',
 'DANF_Demand',
 'DAID_spnDemand',
 'DAIG_spnGeneration',
 'DATF_Demand',
 'SystemTotalLoad_Consumption',
 'DAMargin',
 'DAImbalance',
 'GenerationForecast_Production',
 'Solar_Generation',
 'WindOffshore_Generation',
 'WindOnshore_Generation']

In [ ]:
sel_cols = [x for x in list(df_01_02_imp.columns) if type(x)!=int]
sel_cols

['Weekend',
 'Holiday',
 'Weekday_SIN',
 'Weekday_COS',
 'Month_SIN',
 'Month_COS',
 'Day_SIN',
 'Day_COS',
 'settPeriod_SIN',
 'settPeriod_COS',
 'marketIndexPrice',
 'marketIndexVolume',
 'DANF_Demand',
 'DAID_spnDemand',
 'DAIG_spnGeneration',
 'DATF_Demand',
 'SystemTotalLoad_Consumption',
 'DAMargin',
 'DAImbalance',
 'GenerationForecast_Production',
 'Solar_Generation',
 'WindOffshore_Generation',
 'WindOnshore_Generation']

In [ ]:
df_01_02_imp = df_01_02_imp[sel_cols]

In [ ]:
df_01_02_imp = cat_cols.join(df_01_02_imp)
df_01_02_imp = df_01_02[['settlementDate']].join(df_01_02_imp)
df_01_02_imp.head()

,settlementPeriod,Weekday,Year,Month,Day,Weekend,Holiday,Weekday_SIN,Weekday_COS,Month_SIN,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,1,Sunday,2020,12,20,1.0,0.0,-0.781831,0.62349,-0.5,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,25650.22,0.0,9020.590,7365.717
1,2,Sunday,2020,12,20,1.0,0.0,-0.781831,0.62349,-0.5,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,25229.70,0.0,9020.590,7365.717
2,3,Sunday,2020,12,20,1.0,0.0,-0.781831,0.62349,-0.5,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.00,0.0,8871.388,7352.576
3,4,Sunday,2020,12,20,1.0,0.0,-0.781831,0.62349,-0.5,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.00,0.0,8871.388,7352.576
4,5,Sunday,2020,12,20,1.0,0.0,-0.781831,0.62349,-0.5,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.00,0.0,8670.041,7312.832


In [ ]:
# MISSING VALUES (BY COLUMNS)

df_data = df_01_02_imp.copy()
df_colsnan = pd.DataFrame()

df_colsnan['dtypes'] = df_data.apply(lambda x: x.dtypes)
df_colsnan['nan'] = df_data.isnull().sum()
df_colsnan['perc_nan'] = (df_data.isnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_nan'] = df_colsnan['perc_nan'].round(2)
df_colsnan['not_nan'] = df_data.notnull().sum()
df_colsnan['perc_not_nan'] = (df_data.notnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_not_nan'] = df_colsnan['perc_not_nan'].round(2)
df_colsnan['unique_values'] = df_data.apply(lambda x: len(x.unique()) )
df_colsnan['neg_values'] = df_data.apply(lambda x: x.lt(0).sum() if x.dtypes == 'float64' else 'NA')
df_colsnan.reset_index(inplace=True)
df_colsnan.rename(columns={'index':'col_name'}, inplace=True)

df_colsnan

,col_name,dtypes,nan,perc_nan,not_nan,perc_not_nan,unique_values,neg_values
0,settlementDate,category,0,0.0,44304,100.0,923,NA
1,settlementPeriod,category,0,0.0,44304,100.0,48,NA
2,Weekday,category,0,0.0,44304,100.0,7,NA
3,Year,category,0,0.0,44304,100.0,4,NA
4,Month,category,0,0.0,44304,100.0,12,NA
5,Day,category,0,0.0,44304,100.0,31,NA
6,Weekend,float64,0,0.0,44304,100.0,2,0
7,Holiday,float64,0,0.0,44304,100.0,2,0
8,Weekday_SIN,float64,0,0.0,44304,100.0,7,18960
9,Weekday_COS,float64,0,0.0,44304,100.0,7,25296


In [ ]:
file_path = "C:/Users/FAL/Downloads/Complete Dataframe (IMPUTED).csv"

In [ ]:
df_01_02_imp.to_csv(file_path, index=False)

#### Imputed Data

In [46]:
file_path = '/content/drive/MyDrive/QEnergy/Data/Research/Elexon API/Final Data/Complete Dataframe (IMPUTED).csv'

In [47]:
df_01_02_imp = pd.read_csv(file_path)

In [48]:
df_01_02_imp

,settlementDate,settlementPeriod,Weekday,Year,Month,Day,Weekend,Holiday,Weekday_SIN,Weekday_COS,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,2020-12-20,1,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,25650.22,0.0,9020.590,7365.717
1,2020-12-20,2,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,25229.70,0.0,9020.590,7365.717
2,2020-12-20,3,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.00,0.0,8871.388,7352.576
3,2020-12-20,4,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.00,0.0,8871.388,7352.576
4,2020-12-20,5,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.00,0.0,8670.041,7312.832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-16510.0,26897.0,25613.0,26127.0,21172.0,1284.0,27783.00,0.0,8815.046,4170.762
44300,2023-06-30,45,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-16426.0,25785.0,25123.0,25102.0,21041.0,662.0,27868.00,0.0,9051.941,4506.462
44301,2023-06-30,46,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-15751.0,24347.0,23862.0,23707.0,22087.0,485.0,27445.00,0.0,9051.941,4506.462
44302,2023-06-30,47,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-14830.0,21727.0,22321.0,22499.0,22968.0,-594.0,24550.00,0.0,9152.405,4949.377


In [49]:
# "DATE" COLUMN
df_01_02_imp['settlementDate'] = pd.to_datetime(df_01_02_imp['settlementDate'])

In [50]:
# MISSING VALUES (BY COLUMNS)

df_data = df_01_02_imp.copy()
df_colsnan = pd.DataFrame()

df_colsnan['dtypes'] = df_data.apply(lambda x: x.dtypes)
df_colsnan['nan'] = df_data.isnull().sum()
df_colsnan['perc_nan'] = (df_data.isnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_nan'] = df_colsnan['perc_nan'].round(2)
df_colsnan['not_nan'] = df_data.notnull().sum()
df_colsnan['perc_not_nan'] = (df_data.notnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_not_nan'] = df_colsnan['perc_not_nan'].round(2)
df_colsnan['unique_values'] = df_data.apply(lambda x: len(x.unique()) )
df_colsnan['neg_values'] = df_data.apply(lambda x: x.lt(0).sum() if x.dtypes == 'float64' else 'NA')
df_colsnan.reset_index(inplace=True)
df_colsnan.rename(columns={'index':'col_name'}, inplace=True)

df_colsnan

,col_name,dtypes,nan,perc_nan,not_nan,perc_not_nan,unique_values,neg_values
0,settlementDate,datetime64[ns],0,0.0,44304,100.0,923,NA
1,settlementPeriod,int64,0,0.0,44304,100.0,48,NA
2,Weekday,object,0,0.0,44304,100.0,7,NA
3,Year,int64,0,0.0,44304,100.0,4,NA
4,Month,int64,0,0.0,44304,100.0,12,NA
5,Day,int64,0,0.0,44304,100.0,31,NA
6,Weekend,float64,0,0.0,44304,100.0,2,0
7,Holiday,float64,0,0.0,44304,100.0,2,0
8,Weekday_SIN,float64,0,0.0,44304,100.0,7,18960
9,Weekday_COS,float64,0,0.0,44304,100.0,7,25296


### - Raw Materials prices

Materials and methods

The sample includes all utilities from the Eurozone with available data in Refinitiv (40 firms) and 4 energy raw materials, namely oil, liquefied natural gas (hereafter, gas), coal and uranium. Table 1 shows the proxy variables for each of these energy raw materials.


Energy raw material	Variable proxy:

* Oil	Brent barrel

* Gas	Dutch day-ahead gas price at the Title Transfer Facility (TTF) hub

* Coal	Free on Board (FOB) price at the Vladivistok port

* Uranium	Spot price of U3O8

The empirical study is organized into two stages. The first stage estimates the level of exposure to variations in the price of the energy raw materials, i.e., oil, gas, coal and uranium, based on Jorion's model (1990). Next, considering the estimated exposure levels, the second stage estimates the value at risk of European utility companies to variations in the prices of energy raw materials, considering a weekly time horizon.

https://www.investing.com/commodities/brent-oil-historical-data

https://www.investing.com/commodities/dutch-ttf-gas-c1-futures-historical-data

https://es.investing.com/commodities/newcastle-coal-futures-historical-data

In [51]:
folder_path = '/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data'

In [52]:
desktop = pathlib.Path(folder_path)
# .iterdir() produces a generator
desktop.iterdir()

<generator object Path.iterdir at 0x7b5660f57140>

In [53]:
# Which you can wrap in a list() constructor to materialize
files_list = list(desktop.iterdir())
files_list

[PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/.ipynb_checkpoints'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/Dutch TTF Natural Gas Futures Historical Data.csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/Brent Oil Futures Historical Data.csv'),
 PosixPath('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/Datos históricos Newcastle Coal Futures.csv')]

In [54]:

'''
df_list = []

for x in files_list:
  df_aux = pd.read_csv(x)

  df_list.append(df_aux)

'''

'\ndf_list = []\n\nfor x in files_list:\n  df_aux = pd.read_csv(x)\n\n  df_list.append(df_aux)\n\n'

In [55]:
df_NatGas = pd.read_csv('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/Dutch TTF Natural Gas Futures Historical Data.csv')
df_Coal = pd.read_csv('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/Datos históricos Newcastle Coal Futures.csv')
df_BrentOil = pd.read_csv('/content/drive/MyDrive/QEnergy/Data/Research/Historical Raw Material Prices/Final Data/Brent Oil Futures Historical Data.csv')

In [56]:
df_NatGas = df_NatGas.iloc[:,[0,1]].rename(columns={'Fecha':'Date', 'Price':'NatGas_price'})
df_NatGas['Date'] = pd.to_datetime(df_NatGas['Date'])
df_NatGas['Date_DayAhead'] = df_NatGas['Date'].apply(lambda x: x+dt.timedelta(days=1))
df_NatGas.insert(1, 'Date_DayAhead', df_NatGas.pop('Date_DayAhead'))
df_NatGas = df_NatGas.sort_values(by='Date', ascending=True)
df_NatGas = df_NatGas.reset_index(drop=True)

In [57]:
df_NatGas.head()

,Date,Date_DayAhead,NatGas_price
0,2020-01-02,2020-01-03,12.075
1,2020-01-03,2020-01-04,12.985
2,2020-01-06,2020-01-07,12.280
3,2020-01-07,2020-01-08,11.930
4,2020-01-08,2020-01-09,11.965


In [58]:
df_NatGas[df_NatGas['Date']=='2020-12-18']

,Date,Date_DayAhead,NatGas_price
244,2020-12-18,2020-12-19,15.885


In [59]:
df_Coal = df_Coal.iloc[:,[0,1]].rename(columns={'Fecha':'Date', 'Último':'Coal_price'})
df_Coal['Date'] = pd.to_datetime(df_Coal['Date'])
df_Coal['Date_DayAhead'] = df_Coal['Date'].apply(lambda x: x+dt.timedelta(days=1))
df_Coal['Coal_price'] = df_Coal['Coal_price'].apply(lambda x: x.replace(',','.'))
df_Coal['Coal_price'] = df_Coal['Coal_price'].astype('float')
df_Coal.insert(1, 'Date_DayAhead', df_Coal.pop('Date_DayAhead'))
df_Coal = df_Coal.sort_values(by='Date', ascending=True)
df_Coal = df_Coal.reset_index(drop=True)

<ipython-input-59-db17778e83fa>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_Coal['Date'] = pd.to_datetime(df_Coal['Date'])


In [60]:
df_Coal.head()

,Date,Date_DayAhead,Coal_price
0,2020-01-04,2020-01-05,66.70
1,2020-01-05,2020-01-06,51.25
2,2020-01-06,2020-01-07,55.65
3,2020-01-07,2020-01-08,51.60
4,2020-01-09,2020-01-10,51.40


In [61]:
df_BrentOil = df_BrentOil.iloc[:,[0,1]].rename(columns={'Fecha':'Date', 'Price':'BrentOil_price'})
df_BrentOil['Date'] = pd.to_datetime(df_BrentOil['Date'])
df_BrentOil['Date_DayAhead'] = df_BrentOil['Date'].apply(lambda x: x+dt.timedelta(days=1))
df_BrentOil.insert(1, 'Date_DayAhead', df_BrentOil.pop('Date_DayAhead'))
df_BrentOil = df_BrentOil.sort_values(by='Date', ascending=True)
df_BrentOil = df_BrentOil.reset_index(drop=True)

In [62]:
df_BrentOil.head()

,Date,Date_DayAhead,BrentOil_price
0,2020-01-02,2020-01-03,66.25
1,2020-01-03,2020-01-04,68.60
2,2020-01-06,2020-01-07,68.91
3,2020-01-07,2020-01-08,68.27
4,2020-01-08,2020-01-09,65.44


In [63]:
df_BrentOil[df_BrentOil['Date']=='2020-12-24']

,Date,Date_DayAhead,BrentOil_price
254,2020-12-24,2020-12-25,51.29


In [64]:
df_list = [df_NatGas, df_Coal, df_BrentOil]

df_03 = df_struct.copy()

for x in df_list:

  df_03 = pd.merge(df_03, x, how='left', left_on=['settlementDate'], right_on=['Date_DayAhead'])

df_03 = df_03.loc[:, ~df_03.columns.str.contains("_x")]
df_03 = df_03.loc[:, ~df_03.columns.str.contains("_y")]
df_03 = df_03.loc[:, ~df_03.columns.str.contains("Date_DayAhead")]
df_03 = df_03.drop(columns=['Date'], axis=1)

In [65]:
df_03

,settlementDate,settlementPeriod,NatGas_price,Coal_price,BrentOil_price
0,2020-01-01,1.0,NaN,NaN,NaN
1,2020-01-01,2.0,NaN,NaN,NaN
2,2020-01-01,3.0,NaN,NaN,NaN
3,2020-01-01,4.0,NaN,NaN,NaN
4,2020-01-01,5.0,NaN,NaN,NaN
...,...,...,...,...,...
61291,2023-06-30,44.0,35.18,127.9,74.34
61292,2023-06-30,45.0,35.18,127.9,74.34
61293,2023-06-30,46.0,35.18,127.9,74.34
61294,2023-06-30,47.0,35.18,127.9,74.34


In [66]:
df_03.isnull().sum()

settlementDate          0
settlementPeriod        0
NatGas_price        18960
Coal_price          18912
BrentOil_price      17952
dtype: int64

In [67]:
df_03.loc[765:775,:]

,settlementDate,settlementPeriod,NatGas_price,Coal_price,BrentOil_price
765,2020-01-16,46.0,11.180,70.50,64.00
766,2020-01-16,47.0,11.180,70.50,64.00
767,2020-01-16,48.0,11.180,70.50,64.00
768,2020-01-17,1.0,11.115,70.45,64.62
769,2020-01-17,2.0,11.115,70.45,64.62
770,2020-01-17,3.0,11.115,70.45,64.62
771,2020-01-17,4.0,11.115,70.45,64.62
772,2020-01-17,5.0,11.115,70.45,64.62
773,2020-01-17,6.0,11.115,70.45,64.62
774,2020-01-17,7.0,11.115,70.45,64.62


In [68]:
df_03 = df_03.fillna(method='ffill', axis=0, limit=None)

In [69]:
df_03.isnull().sum()

settlementDate        0
settlementPeriod      0
NatGas_price         96
Coal_price          192
BrentOil_price       96
dtype: int64

In [70]:
df_03.loc[765:775,:]

,settlementDate,settlementPeriod,NatGas_price,Coal_price,BrentOil_price
765,2020-01-16,46.0,11.180,70.50,64.00
766,2020-01-16,47.0,11.180,70.50,64.00
767,2020-01-16,48.0,11.180,70.50,64.00
768,2020-01-17,1.0,11.115,70.45,64.62
769,2020-01-17,2.0,11.115,70.45,64.62
770,2020-01-17,3.0,11.115,70.45,64.62
771,2020-01-17,4.0,11.115,70.45,64.62
772,2020-01-17,5.0,11.115,70.45,64.62
773,2020-01-17,6.0,11.115,70.45,64.62
774,2020-01-17,7.0,11.115,70.45,64.62


In [71]:
# MISSING VALUES (BY COLUMNS)

df_data = df_03.copy()
df_colsnan = pd.DataFrame()

df_colsnan['dtypes'] = df_data.apply(lambda x: x.dtypes)
df_colsnan['nan'] = df_data.isnull().sum()
df_colsnan['perc_nan'] = (df_data.isnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_nan'] = df_colsnan['perc_nan'].round(2)
df_colsnan['not_nan'] = df_data.notnull().sum()
df_colsnan['perc_not_nan'] = (df_data.notnull().sum() / df_data.shape[0]) * 100
df_colsnan['perc_not_nan'] = df_colsnan['perc_not_nan'].round(2)
df_colsnan['unique_values'] = df_data.apply(lambda x: len(x.unique()) )
df_colsnan['neg_values'] = df_data.apply(lambda x: x.lt(0).sum() if x.dtypes == 'float64' else 'NA')
df_colsnan.reset_index(inplace=True)
df_colsnan.rename(columns={'index':'col_name'}, inplace=True)

df_colsnan

,col_name,dtypes,nan,perc_nan,not_nan,perc_not_nan,unique_values,neg_values
0,settlementDate,datetime64[ns],0,0.00,61296,100.00,1277,NA
1,settlementPeriod,float64,0,0.00,61296,100.00,48,0
2,NatGas_price,float64,96,0.16,61200,99.84,857,0
3,Coal_price,float64,192,0.31,61104,99.69,700,0
4,BrentOil_price,float64,96,0.16,61200,99.84,853,0


In [72]:
df_03

,settlementDate,settlementPeriod,NatGas_price,Coal_price,BrentOil_price
0,2020-01-01,1.0,NaN,NaN,NaN
1,2020-01-01,2.0,NaN,NaN,NaN
2,2020-01-01,3.0,NaN,NaN,NaN
3,2020-01-01,4.0,NaN,NaN,NaN
4,2020-01-01,5.0,NaN,NaN,NaN
...,...,...,...,...,...
61291,2023-06-30,44.0,35.18,127.9,74.34
61292,2023-06-30,45.0,35.18,127.9,74.34
61293,2023-06-30,46.0,35.18,127.9,74.34
61294,2023-06-30,47.0,35.18,127.9,74.34


In [73]:
df_03 = df_03[df_03['settlementDate']>='2020-12-20']
df_03.reset_index(inplace=True, drop=True)
df_03

,settlementDate,settlementPeriod,NatGas_price,Coal_price,BrentOil_price
0,2020-12-20,1.0,15.885,80.0,52.26
1,2020-12-20,2.0,15.885,80.0,52.26
2,2020-12-20,3.0,15.885,80.0,52.26
3,2020-12-20,4.0,15.885,80.0,52.26
4,2020-12-20,5.0,15.885,80.0,52.26
...,...,...,...,...,...
44299,2023-06-30,44.0,35.180,127.9,74.34
44300,2023-06-30,45.0,35.180,127.9,74.34
44301,2023-06-30,46.0,35.180,127.9,74.34
44302,2023-06-30,47.0,35.180,127.9,74.34


## Preparing complete dataset

In [74]:
df_01_02_imp

,settlementDate,settlementPeriod,Weekday,Year,Month,Day,Weekend,Holiday,Weekday_SIN,Weekday_COS,...,DAID_spnDemand,DAIG_spnGeneration,DATF_Demand,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation
0,2020-12-20,1,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-20445.0,24059.0,24160.0,25732.0,27928.0,-101.0,25650.22,0.0,9020.590,7365.717
1,2020-12-20,2,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-20591.0,23904.0,23919.0,25545.0,27965.0,-16.0,25229.70,0.0,9020.590,7365.717
2,2020-12-20,3,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-19622.0,23298.0,22964.0,24831.0,28570.0,334.0,24697.00,0.0,8871.388,7352.576
3,2020-12-20,4,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-19358.0,22976.0,22512.0,24006.0,28927.0,464.0,24499.00,0.0,8871.388,7352.576
4,2020-12-20,5,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,-19002.0,22509.0,22041.0,23500.0,29006.0,468.0,24337.00,0.0,8670.041,7312.832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-16510.0,26897.0,25613.0,26127.0,21172.0,1284.0,27783.00,0.0,8815.046,4170.762
44300,2023-06-30,45,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-16426.0,25785.0,25123.0,25102.0,21041.0,662.0,27868.00,0.0,9051.941,4506.462
44301,2023-06-30,46,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-15751.0,24347.0,23862.0,23707.0,22087.0,485.0,27445.00,0.0,9051.941,4506.462
44302,2023-06-30,47,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,-14830.0,21727.0,22321.0,22499.0,22968.0,-594.0,24550.00,0.0,9152.405,4949.377


In [75]:
df_01_02_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44304 entries, 0 to 44303
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   settlementDate                 44304 non-null  datetime64[ns]
 1   settlementPeriod               44304 non-null  int64         
 2   Weekday                        44304 non-null  object        
 3   Year                           44304 non-null  int64         
 4   Month                          44304 non-null  int64         
 5   Day                            44304 non-null  int64         
 6   Weekend                        44304 non-null  float64       
 7   Holiday                        44304 non-null  float64       
 8   Weekday_SIN                    44304 non-null  float64       
 9   Weekday_COS                    44304 non-null  float64       
 10  Month_SIN                      44304 non-null  float64       
 11  Month_COS      

In [76]:
df_03

,settlementDate,settlementPeriod,NatGas_price,Coal_price,BrentOil_price
0,2020-12-20,1.0,15.885,80.0,52.26
1,2020-12-20,2.0,15.885,80.0,52.26
2,2020-12-20,3.0,15.885,80.0,52.26
3,2020-12-20,4.0,15.885,80.0,52.26
4,2020-12-20,5.0,15.885,80.0,52.26
...,...,...,...,...,...
44299,2023-06-30,44.0,35.180,127.9,74.34
44300,2023-06-30,45.0,35.180,127.9,74.34
44301,2023-06-30,46.0,35.180,127.9,74.34
44302,2023-06-30,47.0,35.180,127.9,74.34


In [77]:
df_complete = pd.merge(df_01_02_imp, df_03,  how='left', on=['settlementDate', 'settlementPeriod'])
#df_complete = df_complete.drop(columns=['Fecha'], axis=1)
#df_complete.insert(1, 'settlementPeriod', df_complete.pop('settlementPeriod'))
df_complete

,settlementDate,settlementPeriod,Weekday,Year,Month,Day,Weekend,Holiday,Weekday_SIN,Weekday_COS,...,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price
0,2020-12-20,1,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,25732.0,27928.0,-101.0,25650.22,0.0,9020.590,7365.717,15.885,80.0,52.26
1,2020-12-20,2,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,25545.0,27965.0,-16.0,25229.70,0.0,9020.590,7365.717,15.885,80.0,52.26
2,2020-12-20,3,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,24831.0,28570.0,334.0,24697.00,0.0,8871.388,7352.576,15.885,80.0,52.26
3,2020-12-20,4,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,24006.0,28927.0,464.0,24499.00,0.0,8871.388,7352.576,15.885,80.0,52.26
4,2020-12-20,5,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,23500.0,29006.0,468.0,24337.00,0.0,8670.041,7312.832,15.885,80.0,52.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,26127.0,21172.0,1284.0,27783.00,0.0,8815.046,4170.762,35.180,127.9,74.34
44300,2023-06-30,45,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,25102.0,21041.0,662.0,27868.00,0.0,9051.941,4506.462,35.180,127.9,74.34
44301,2023-06-30,46,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,23707.0,22087.0,485.0,27445.00,0.0,9051.941,4506.462,35.180,127.9,74.34
44302,2023-06-30,47,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,22499.0,22968.0,-594.0,24550.00,0.0,9152.405,4949.377,35.180,127.9,74.34


In [78]:
file_path = '/content/drive/MyDrive/QEnergy/Data/Research/Research Data.csv'

In [79]:
#df_complete.to_csv(file_path, index=False)

In [80]:
df_complete = pd.read_csv(file_path)

In [81]:
# "DATE" COLUMN
df_complete['settlementDate'] = pd.to_datetime(df_complete['settlementDate'])

In [82]:
df_complete

,settlementDate,settlementPeriod,Weekday,Year,Month,Day,Weekend,Holiday,Weekday_SIN,Weekday_COS,...,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price
0,2020-12-20,1,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,25732.0,27928.0,-101.0,25650.22,0.0,9020.590,7365.717,15.885,80.0,52.26
1,2020-12-20,2,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,25545.0,27965.0,-16.0,25229.70,0.0,9020.590,7365.717,15.885,80.0,52.26
2,2020-12-20,3,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,24831.0,28570.0,334.0,24697.00,0.0,8871.388,7352.576,15.885,80.0,52.26
3,2020-12-20,4,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,24006.0,28927.0,464.0,24499.00,0.0,8871.388,7352.576,15.885,80.0,52.26
4,2020-12-20,5,Sunday,2020,12,20,1.0,0.0,-0.781831,0.623490,...,23500.0,29006.0,468.0,24337.00,0.0,8670.041,7312.832,15.885,80.0,52.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44299,2023-06-30,44,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,26127.0,21172.0,1284.0,27783.00,0.0,8815.046,4170.762,35.180,127.9,74.34
44300,2023-06-30,45,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,25102.0,21041.0,662.0,27868.00,0.0,9051.941,4506.462,35.180,127.9,74.34
44301,2023-06-30,46,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,23707.0,22087.0,485.0,27445.00,0.0,9051.941,4506.462,35.180,127.9,74.34
44302,2023-06-30,47,Friday,2023,6,30,0.0,0.0,-0.433884,-0.900969,...,22499.0,22968.0,-594.0,24550.00,0.0,9152.405,4949.377,35.180,127.9,74.34


In [83]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44304 entries, 0 to 44303
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   settlementDate                 44304 non-null  datetime64[ns]
 1   settlementPeriod               44304 non-null  int64         
 2   Weekday                        44304 non-null  object        
 3   Year                           44304 non-null  int64         
 4   Month                          44304 non-null  int64         
 5   Day                            44304 non-null  int64         
 6   Weekend                        44304 non-null  float64       
 7   Holiday                        44304 non-null  float64       
 8   Weekday_SIN                    44304 non-null  float64       
 9   Weekday_COS                    44304 non-null  float64       
 10  Month_SIN                      44304 non-null  float64       
 11  Month_COS      

In [84]:
df_complete[df_complete['settlementDate']=='2021-01-04']

,settlementDate,settlementPeriod,Weekday,Year,Month,Day,Weekend,Holiday,Weekday_SIN,Weekday_COS,...,SystemTotalLoad_Consumption,DAMargin,DAImbalance,GenerationForecast_Production,Solar_Generation,WindOffshore_Generation,WindOnshore_Generation,NatGas_price,Coal_price,BrentOil_price
720,2021-01-04,1,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,27152.00,32723.00,-632.00,25634.00,0.0,6811.26700,2417.88000,19.125,85.6,51.8
721,2021-01-04,2,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,27526.00,32974.00,-367.00,25685.00,0.0,6811.26700,2417.88000,19.125,85.6,51.8
722,2021-01-04,3,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,27082.00,33144.00,-236.00,25681.00,0.0,6874.74700,2449.25400,19.125,85.6,51.8
723,2021-01-04,4,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,26540.00,33083.00,-525.00,25585.00,0.0,6874.74700,2449.25400,19.125,85.6,51.8
724,2021-01-04,5,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,26346.00,32943.00,-601.00,25320.00,0.0,6945.39900,2436.38200,19.125,85.6,51.8
725,2021-01-04,6,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,26177.00,32659.00,-704.00,25427.00,0.0,6945.39900,2436.38200,19.125,85.6,51.8
726,2021-01-04,7,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,25872.00,32999.00,-492.00,25534.00,0.0,7099.13300,2436.66500,19.125,85.6,51.8
727,2021-01-04,8,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,25576.00,33457.00,26.00,25650.00,0.0,7099.13300,2436.66500,19.125,85.6,51.8
728,2021-01-04,9,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,25441.00,33673.00,257.00,25726.00,0.0,7284.44900,2512.31200,19.125,85.6,51.8
729,2021-01-04,10,Monday,2021,1,4,0.0,0.0,0.0,1.0,...,25797.00,33703.00,344.00,25808.00,0.0,7284.44900,2512.31200,19.125,85.6,51.8
